In [1]:
import pytest
import torch

from nerfacc import OccupancyGrid, ray_marching, unpack_info

import taichi as ti
ti.init(arch=ti.cuda)

/home/loyot/anaconda3/envs/nerfacc/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Taichi] version 1.2.1, llvm 10.0.0, commit 12ab828a, linux, python 3.9.13
[I 11/03/22 13:59:17.590 11951] [shell.py:_shell_pop_print@33] Graphical python shell detected, using wrapped sys.stdout
[Taichi] Starting on arch=cuda


In [2]:
device = "cuda:0"
batch_size = 128

In [3]:
def test_marching_with_grid_cuda():
    rays_o = torch.rand((batch_size, 3), device=device)
    rays_d = torch.randn((batch_size, 3), device=device)
    rays_d = rays_d / rays_d.norm(dim=-1, keepdim=True)
    grid = OccupancyGrid(roi_aabb=[0, 0, 0, 1, 1, 1]).to(device)
    grid._binary[:] = True

    packed_info, t_starts, t_ends = ray_marching(
        rays_o,
        rays_d,
        grid=grid,
        near_plane=0.0,
        far_plane=1.0,
        render_step_size=1e-2,
    )
    torch.cuda.synchronize()

In [7]:
%timeit test_marching_with_grid_cuda() # grid_binary.flatten() with block

13.8 ms ± 25.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit test_marching_with_grid_cuda() # packed_info.flatten() t_starts.flatten() t_ends.flatten()

14.5 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit test_marching_with_grid_cuda() # no block

14.4 ms ± 90.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
%timeit test_marching_with_grid_cuda() # grid_binary shape 1

14.7 ms ± 325 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [7]:
%timeit test_marching_with_grid_cuda() #cuda

13.9 ms ± 66.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
